# Pratica: autenticação de usuários 
- Registrar um novo usuário.
- Login de usuário (gerando um token JWT simples).
- Obter informações do usuário logado (protegido por token).

In [ ]:
# importações - instalar
!pip install pydantic passlib[bcrypt] python-jose[cryptography]

In [26]:
# Importações
import enum
import hashlib
from typing import Optional, Any, Dict

from pydantic import BaseModel, Field, EmailStr, model_validator
from passlib.context import CryptContext

In [27]:
#Configuração de segurança para o hashing de senha
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

In [28]:
# Distribuição dos papéis
class Role(enum.IntFlag):
    User = 0
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8
    
class User(BaseModel):
    id: Optional[int] = None
    name: str = Field(..., min_length=3, max_length=50, description="Nome do usuário")
    email: EmailStr = Field(..., description="Email válido do usuário")
    password_hash: str # O hash da senha, nunca a senha em texto puro
    role: Role = Role.User # Papel do usuário, padrão é User

    # Decorador para validar dados ANTES de criar o modelo
    @model_validator(mode="before")
    @classmethod
    def validate_and_hash_password(cls, values: Dict[str, Any]) -> Dict[str, Any]:
        # Validação de campo "password"
        if "password" not in values or not values["password"]:
            raise ValueError("Senha é obrigatória.")

        password = values["password"]
        name = values.get("name", "") # Pega o nome, se existir

        # Validação de complexidade da senha
        if len(password) < 8 or not any(char.isupper() for char in password) or \
           not any(char.islower() for char in password) or not any(char.isdigit() for char in password):
            raise ValueError("A senha deve ter no mínimo 8 caracteres, 1 maiúscula, 1 minúscula e 1 número.")
        
        # Validação de nome na senha (case-insensitive)
        if name and name.casefold() in password.casefold():
            raise ValueError("A senha não pode conter o nome do usuário.")

        # HAsh da senha antes de atribuir ao password_hash
        values["password_hash"] = pwd_context.hash(password)
        # Remove a senha original do dicionário para não ser armazenada ou exposta
        del values["password"]
        return values


In [ ]:
#Funçãop de demonstração
def demonstrate_user_model_features() -> None:

    # Caso de Sucesso: Dados Válidos
    print("\n-> Caso 1: Dados Válidos de Usuário")
    good_data = {
        "id": 1,
        "name": "Arjan",
        "email": "example@arjancodes.com",
        "password": "Password123ABC", # Senha válida
        "role": "Admin",
    }
    
    try:
        user_valid = User.model_validate(good_data)
        print("Usuário Validado com Sucesso!")
        print(
            "Serialização para Dicionário:",
            user_valid.model_dump(),
            sep="\n",
            end="\n\n",
        )
        print(
            "Serialização para String JSON:",
            user_valid.model_dump(mode="json"),
            sep="\n",
            end="\n\n",
        )
        print(
            "Serialização para JSON (excluindo 'role'):",
            user_valid.model_dump(exclude=["role"], mode="json"),
            sep="\n",
            end="\n\n",
        )
        print("Conversão direta para Dicionário:", dict(user_valid), sep="\n")
        print(f"\nVerificando a senha 'Password123ABC': {user_valid.verify_password('Password123ABC')}")
        print(f"Role: {user_valid.role.name} (Valor: {user_valid.role.value})")
        print(f"É Admin? {Role.Admin in user_valid.role}")
        print("-" * 50)

    except Exception as e:
        print(f"Erro inesperado no caso válido: {e}")
        print("-" * 50)


    # Caso de Falha: Senha Fraca 
    print("\n-> Caso 2: Senha Fraca (menor que 8 chars)")
    weak_password_data = {
        "name": "João",
        "email": "joao@example.com",
        "password": "123", # Senha fraca
        "role": "User",
    }
    try:
        User.model_validate(weak_password_data)
        print("ERRO: Usuário com senha fraca foi validado!")
    except ValueError as e:
        print("Sucesso! Erro de validação da senha fraca:", e)
    except Exception as e:
        print(f"Erro inesperado: {e}")
    print("-" * 50)

    # Caso de Falha: Senha Contém Nome 
    print("\n-> Caso 3: Senha Contém o Nome do Usuário")
    password_with_name_data = {
        "name": "Carlos",
        "email": "carlos@example.com",
        "password": "Carlos123", # Senha contém o nome
        "role": "User",
    }
    try:
        User.model_validate(password_with_name_data)
        print("ERRO: Usuário com senha contendo nome foi validado!")
    except ValueError as e:
        print("Sucesso! Erro de validação: senha contém o nome:", e)
    except Exception as e:
        print(f"Erro inesperado: {e}")
    print("-" * 50)

    # Caso de Falha: Email Inválido (validação do Pydantic) 
    print("\n-> Caso 4: Email Inválido ")
    invalid_email_data = {
        "name": "Maria",
        "email": "maria@.com", # Email inválido
        "password": "SenhaForte123",
        "role": "Editor",
    }
    try:
        User.model_validate(invalid_email_data)
        print("ERRO: Usuário com email inválido foi validado!")
    except Exception as e: # Pydantic levanta ValidationError aqui, que é subclasse de Exception
        print("Sucesso! Erro de validação de email:", e)
    print("-" * 50)

In [30]:
demonstrate_user_model_features() # Executa a demonstração

(trapped) error reading bcrypt version
Traceback (most recent call last):
  File "/home/guimileib/Área de trabalho/FastCamp/card2/.venv/lib/python3.12/site-packages/passlib/handlers/bcrypt.py", line 620, in _load_backend_mixin
    version = _bcrypt.__about__.__version__
              ^^^^^^^^^^^^^^^^^
AttributeError: module 'bcrypt' has no attribute '__about__'


--- DEMONSTRANDO AS FUNCIONALIDADES DO MODELO USER ---

### Caso 1: Dados Válidos de Usuário ###
Erro inesperado no caso válido: 1 validation error for User
role
  Input should be 0, 1, 2, 4 or 8 [type=enum, input_value='Admin', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/enum
--------------------------------------------------

### Caso 2: Senha Fraca (menor que 8 chars) ###
Sucesso! Erro de validação da senha fraca: 1 validation error for User
  Value error, A senha deve ter no mínimo 8 caracteres, 1 maiúscula, 1 minúscula e 1 número. [type=value_error, input_value={'name': 'João', 'email'...: '123', 'role': 'User'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error
--------------------------------------------------

### Caso 3: Senha Contém o Nome do Usuário ###
Sucesso! Erro de validação: senha contém o nome: 1 validation error for User
  Value error, A senha não pode conter o nome do usuário. [